In [1]:
R.version.string

[1] "R version 4.2.0 (2022-04-22)"

```shell
cd /home/xxzhang/workplace/software/ #切入到安装目录
```

```shell
wget https://ftp.ncbi.nlm.nih.gov/genomes/TOOLS/ORFfinder/linux-i64/ORFfinder.gz
wget https://ftp.ncbi.nlm.nih.gov/genomes/TOOLS/ORFfinder/CHANGELOG.txt
wget https://ftp.ncbi.nlm.nih.gov/genomes/TOOLS/ORFfinder/FASTA_example.fsa
wget https://ftp.ncbi.nlm.nih.gov/genomes/TOOLS/ORFfinder/ORFfinder.asn_spec.txt
./ORFfinder -h
```
> ./ORFfinder: error while loading shared libraries: libuv.so.1: cannot open shared object file: No such file or directory \n
> 我现在遇到的这个问题是，我不知道非root用户，如何安装系统库libuv.so。 \n
> 可以使用conda来安装吗？ \n
> 我也不了解原理，我想先试一下(在base环境下)。
```shell
conda install conda-forge::libuv
```
> 即，虽然感觉安装了，但是并没有什么用处。没找对解决方案；
> which  查看可执行文件的位置。
> whereis 查看文件的位置。 
> 
find   实际搜寻硬盘查询文件名称

最后使用指令“export LD_LIBRARY_PATH=\$CONDA_PREFIX/lib:\$LD_LIBRARY_PATH”
并且创建了一个新的conda环境，两者相结合，可以实现在新的conda环境下，运行ORFfinder；
（我真的懒得写很详细的归纳的文档了）。

In [1]:
setwd("/home/xxzhang/workplace/software/ORFFinder/")

In [2]:
data<-read.table("query-hg38_bedtools_L1.chr1-Y.notAnnotation.txt")

In [3]:
colnames(data)<-c("query.acc.ver","subject.acc.ver", "%identity", 
                  "alignment.length",
                  "mismatches","gap.opens","q.start",
                  "q.end","s.start","s. end","evalue","bit.score")

In [4]:
head(data)

,query.acc.ver,subject.acc.ver,%identity,alignment.length,mismatches,gap.opens,q.start,q.end,s.start,s. end,evalue,bit.score
,<chr>,<chr>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>
1,lcl|L1PA2::chr1:73845-74895(-):9-833,Human-LINEl-mRNA-ORF2|M19503.1:1957-5784,97.939,825,17,0,1,825,3004,3828,0,1430
2,lcl|L1PA2::chr1:74898-76697(+):83-1798,Human-LINEl-mRNA-ORF2|M19503.1:1957-5784,97.021,1712,51,0,1,1712,1285,2996,0,2887
3,lcl|L1PA3::chr1:2364327-2365707(-):363-1379,Human-LINEl-mRNA-ORF2|M19503.1:1957-5784,95.805,1025,34,3,1,1016,2260,3284,0,1652
4,lcl|L1PA3::chr1:4102544-4105048(+):569-2248,Human-LINEl-mRNA-ORF2|M19503.1:1957-5784,95.595,1680,70,2,1,1677,2105,3783,0,2697
5,lcl|L1PA3::chr1:4244385-4247142(-):824-2188,Human-LINEl-mRNA-ORF2|M19503.1:1957-5784,96.193,1366,47,3,1,1362,2110,3474,0,2237
6,lcl|L1PA4::chr1:7411758-7417922(-):2123-3226,Human-LINEl-mRNA-ORF2|M19503.1:1957-5784,97.003,1101,33,0,1,1101,1,1101,0,1851


In [5]:
library(dplyr) #loading packages
library(tidyr)
library(ggplot2)


载入程辑包：‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [6]:
#library(BiocManager)

In [7]:
data_ext<-data %>% separate(query.acc.ver, c("lcl","subfamily","null","chromsome","start-stop","strand","orf.start-orf.end"),sep = "[:|(]")

In [8]:
#head(data_ext)

In [9]:
data_ext2<-data_ext %>% separate(`start-stop`, c("start","stop"),sep = "[-]")
data_ext3<-data_ext2 %>% separate(`orf.start-orf.end`, c("orf.start","orf.end"),sep = "[-]")
data_ext3[which(data_ext3$strand=="+)"),]$strand<-"+"
data_ext3[which(data_ext3$strand=="-)"),]$strand<-"-"
data_ext3[grepl("ORF2",data_ext3$subject.acc.ver),"ORF"]<-"ORF2"
data_ext3[grepl("ORF1",data_ext3$subject.acc.ver),"ORF"]<-"ORF1"
#data_ext3[which(data_ext3$subject.acc.ver=="Human-LINEl-mRNA-ORF1|M19503.1:877-1893"),]$ORF<-"ORF1"
data_ext3$Length<-as.numeric(data_ext3$stop)-as.numeric(data_ext3$start)

In [10]:
#table(data_ext3$subject.acc.ver)

In [11]:
data_ext3[1,10:21]

,subject.acc.ver,%identity,alignment.length,mismatches,gap.opens,q.start,q.end,s.start,s. end,evalue,bit.score,ORF
,<chr>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<chr>
1,Human-LINEl-mRNA-ORF2|M19503.1:1957-5784,97.939,825,17,0,1,825,3004,3828,0,1430,ORF2


In [90]:
data_ext4<-data_ext3[,c(2,4,5,6,7,22,8,9,11,12,13,14,15,16,17,18,21)] #从这边开始，重新做

In [91]:
head(data_ext4,10)

,subfamily,chromsome,start,stop,strand,Length,orf.start,orf.end,%identity,alignment.length,mismatches,gap.opens,q.start,q.end,s.start,s. end,ORF
,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<chr>
1,L1PA2,chr1,73845,74895,-,1050,9,833,97.939,825,17,0,1,825,3004,3828,ORF2
2,L1PA2,chr1,74898,76697,+,1799,83,1798,97.021,1712,51,0,1,1712,1285,2996,ORF2
3,L1PA3,chr1,2364327,2365707,-,1380,363,1379,95.805,1025,34,3,1,1016,2260,3284,ORF2
4,L1PA3,chr1,4102544,4105048,+,2504,569,2248,95.595,1680,70,2,1,1677,2105,3783,ORF2
5,L1PA3,chr1,4244385,4247142,-,2757,824,2188,96.193,1366,47,3,1,1362,2110,3474,ORF2
6,L1PA4,chr1,7411758,7417922,-,6164,2123,3226,97.003,1101,33,0,1,1101,1,1101,ORF2
7,L1PA4,chr1,7411758,7417922,-,6164,3729,4991,96.038,1262,50,0,1,1262,1606,2867,ORF2
8,L1PA4,chr1,7597938,7600485,-,2547,762,1967,94.601,1204,62,2,1,1204,2260,3460,ORF2
9,L1PA3,chr1,11083624,11086544,-,2920,166,1140,96.134,957,37,0,1,957,1285,2241,ORF2


In [92]:
data_ext4[,"label"]<-"null"

In [93]:
data_ext4<-data_ext4[order(data_ext4$subfamily,data_ext4$chromsome,data_ext4$start,
                           data_ext4$stop,data_ext4$strand,data_ext4$Length,data_ext4$ORF),] #加上这一行重新开始做；

In [94]:
dim(data_ext4)

[1] 15201    18

In [96]:
data_ext4[data_ext4$start=="77470642",]

,subfamily,chromsome,start,stop,strand,Length,orf.start,orf.end,%identity,alignment.length,mismatches,gap.opens,q.start,q.end,s.start,s. end,ORF,label
,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<chr>,<chr>
14262,L1PA5,chrX,77470642,77476783,+,6141,771,2033,89.577,1017,106,0,247,1263,1,1017,ORF1,null
14261,L1PA5,chrX,77470642,77476783,+,6141,3929,5317,94.831,1393,68,1,1,1389,1837,3229,ORF2,null
14263,L1PA5,chrX,77470642,77476783,+,6141,2097,2957,93.503,862,53,1,1,859,1,862,ORF2,null


In [97]:
#merge lines
for (i in 1:(dim(data_ext4)[1]-1)) {
if(sum(data_ext4[i,c(1:6,17)]==data_ext4[i+1,c(1:6,17)])==7){
    a=c(data_ext4[i,7],data_ext4[i,8],data_ext4[i+1,7],data_ext4[i+1,8])
    data_ext4[i,]$label<-"1rd"
    data_ext4[i+1,]$label<-"1rd"
    b=as.data.frame(c(data_ext4[i,1:6],min(a),max(a),rep(NA,8),data_ext4[i,17],"2rd"))
    colnames(b)<-colnames(data_ext4)
    data_ext4<-rbind(data_ext4,b)
    i = i+1
}
    }
#这里就不是很会写了，感觉两个相邻行还比较好处理，如果多个这样要怎么办呢？要不断的这样循环的迭代嘛？
#而且这里当时发现，对于最后一行是没有下一行的，因此难免会报出错误；之前运行到这里的时候，居然没有报错也是非常的奇怪

In [98]:
data_ext4[c(15200,15201),] #最后一行和上一行并不重复，不需要额外考虑；

,subfamily,chromsome,start,stop,strand,Length,orf.start,orf.end,%identity,alignment.length,mismatches,gap.opens,q.start,q.end,s.start,s. end,ORF,label
,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<chr>,<chr>
7421,L1PREC2,chr3,110671440,110676505,+,5065,2684,3754,91.511,1072,90,1,1,1071,427,1498,ORF2,null
14181,L1PREC2,chrX,74758417,74760841,-,2424,248,1057,83.106,805,129,5,1,801,1837,2638,ORF2,null


In [99]:
#data_ext4[1,1:6]==data_ext4[2,1:6]

In [100]:
#head(data_ext4,20)
data_ext4[data_ext4$start=="7411758",]
data_ext4[data_ext4$start=="3160921",]

,subfamily,chromsome,start,stop,strand,Length,orf.start,orf.end,%identity,alignment.length,mismatches,gap.opens,q.start,q.end,s.start,s. end,ORF,label
,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<chr>,<chr>
6,L1PA4,chr1,7411758,7417922,-,6164,2123,3226,97.003,1101,33,0,1,1101,1,1101,ORF2,1rd
7,L1PA4,chr1,7411758,7417922,-,6164,3729,4991,96.038,1262,50,0,1,1262,1606,2867,ORF2,1rd
17303,L1PA4,chr1,7411758,7417922,-,6164,2123,4991,NA,NA,NA,NA,NA,NA,NA,NA,ORF2,2rd


,subfamily,chromsome,start,stop,strand,Length,orf.start,orf.end,%identity,alignment.length,mismatches,gap.opens,q.start,q.end,s.start,s. end,ORF,label
,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<chr>,<chr>
7917,L1PA7,chr4,3160921,3167073,-,6152,1034,1849,83.111,823,116,7,4,813,1,813,ORF1,null
7916,L1PA7,chr4,3160921,3167073,-,6152,2308,3231,90.409,928,77,4,1,922,197,1118,ORF2,1rd
7918,L1PA7,chr4,3160921,3167073,-,6152,5090,5935,90.307,846,82,0,1,846,2983,3828,ORF2,1rd
18212,L1PA7,chr4,3160921,3167073,-,6152,2308,5935,NA,NA,NA,NA,NA,NA,NA,NA,ORF2,2rd


In [101]:
data_ext5<-data_ext4[!grepl("1rd", data_ext4$label),] #删除1rd的结果

In [103]:
data_ext5[data_ext5$start=="7411758",] #到这一步是没有问题的;
data_ext5[data_ext5$start=="3160921",]

,subfamily,chromsome,start,stop,strand,Length,orf.start,orf.end,%identity,alignment.length,mismatches,gap.opens,q.start,q.end,s.start,s. end,ORF,label
,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<chr>,<chr>
17303,L1PA4,chr1,7411758,7417922,-,6164,2123,4991,NA,NA,NA,NA,NA,NA,NA,NA,ORF2,2rd


,subfamily,chromsome,start,stop,strand,Length,orf.start,orf.end,%identity,alignment.length,mismatches,gap.opens,q.start,q.end,s.start,s. end,ORF,label
,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<chr>,<chr>
7917,L1PA7,chr4,3160921,3167073,-,6152,1034,1849,83.111,823,116,7,4,813,1,813,ORF1,null
18212,L1PA7,chr4,3160921,3167073,-,6152,2308,5935,NA,NA,NA,NA,NA,NA,NA,NA,ORF2,2rd


In [104]:
data_ext5[data_ext5$start=="77470642",]

,subfamily,chromsome,start,stop,strand,Length,orf.start,orf.end,%identity,alignment.length,mismatches,gap.opens,q.start,q.end,s.start,s. end,ORF,label
,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<chr>,<chr>
14262,L1PA5,chrX,77470642,77476783,+,6141,771,2033,89.577,1017,106,0,247,1263,1,1017,ORF1,null
18112,L1PA5,chrX,77470642,77476783,+,6141,2097,5317,NA,NA,NA,NA,NA,NA,NA,NA,ORF2,2rd


In [29]:
#tail(data_ext5,20) #在2nd这边，依然可以看到一些重复，比如17060921

In [105]:
data_ext5[data_ext5$start=="17060921",] #考虑的是相邻的；

,subfamily,chromsome,start,stop,strand,Length,orf.start,orf.end,%identity,alignment.length,mismatches,gap.opens,q.start,q.end,s.start,s. end,ORF,label
,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<chr>,<chr>
15113,L1PA2,chrY,17060921,17066963,-,6042,906,1922,97.64,1017,24,0,1,1017,1,1017,ORF1,null
16172,L1PA2,chrY,17060921,17066963,-,6042,1986,4775,NA,NA,NA,NA,NA,NA,NA,NA,ORF2,2rd
16173,L1PA2,chrY,17060921,17066963,-,6042,3603,5825,NA,NA,NA,NA,NA,NA,NA,NA,ORF2,2rd


In [106]:
lines<-c() #这里还是有点问题，有些行没有被记录上

In [107]:
dim(data_ext5)[1]

[1] 12453

In [108]:
data_ext5[c(12452,12453),]

,subfamily,chromsome,start,stop,strand,Length,orf.start,orf.end,%identity,alignment.length,mismatches,gap.opens,q.start,q.end,s.start,s. end,ORF,label
,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<chr>,<chr>
18226,L1PA8,chr8,114580379,114586371,-,5992,1987,5153,NA,NA,NA,NA,NA,NA,NA,NA,ORF2,2rd
18227,L1PA8,chr9,109660208,109666322,-,6114,2290,4618,NA,NA,NA,NA,NA,NA,NA,NA,ORF2,2rd


In [109]:
for (j in 1:(dim(data_ext5)[1]-1)) {
if(sum(data_ext5[j,c(1:6,17,18)]==data_ext5[j+1,c(1:6,17,18)])==8){
    lines<-append(lines,j) #有一点小聪明
    lines<-append(lines,j+1)
    a=c(data_ext5[j,7],data_ext5[j,8],data_ext5[j+1,7],data_ext5[j+1,8])
    #data_ext5[i,]$label<-"1rd"
    #data_ext5[i+1,]$label<-"1rd"
    b=as.data.frame(c(data_ext5[j,1:6],min(a),max(a),rep(NA,8),data_ext5[j,17],"3rd"))
    colnames(b)<-colnames(data_ext5)
    data_ext5<-rbind(data_ext5,b)
    j = j+1
}
    }

In [110]:
data_ext5[which(data_ext5$start=="17060921",arr.ind=TRUE),] #考虑的是相邻的；
which(data_ext5$start=="17060921",arr.ind=TRUE)

,subfamily,chromsome,start,stop,strand,Length,orf.start,orf.end,%identity,alignment.length,mismatches,gap.opens,q.start,q.end,s.start,s. end,ORF,label
,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<chr>,<chr>
15113,L1PA2,chrY,17060921,17066963,-,6042,906,1922,97.64,1017,24,0,1,1017,1,1017,ORF1,null
16172,L1PA2,chrY,17060921,17066963,-,6042,1986,4775,NA,NA,NA,NA,NA,NA,NA,NA,ORF2,2rd
16173,L1PA2,chrY,17060921,17066963,-,6042,3603,5825,NA,NA,NA,NA,NA,NA,NA,NA,ORF2,2rd
1244,L1PA2,chrY,17060921,17066963,-,6042,1986,5825,NA,NA,NA,NA,NA,NA,NA,NA,ORF2,3rd


[1]  2673 10398 10399 12556

In [111]:
#"12562"%in%lines

In [112]:
data_ext5[data_ext5$start=="17060921",]

,subfamily,chromsome,start,stop,strand,Length,orf.start,orf.end,%identity,alignment.length,mismatches,gap.opens,q.start,q.end,s.start,s. end,ORF,label
,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<chr>,<chr>
15113,L1PA2,chrY,17060921,17066963,-,6042,906,1922,97.64,1017,24,0,1,1017,1,1017,ORF1,null
16172,L1PA2,chrY,17060921,17066963,-,6042,1986,4775,NA,NA,NA,NA,NA,NA,NA,NA,ORF2,2rd
16173,L1PA2,chrY,17060921,17066963,-,6042,3603,5825,NA,NA,NA,NA,NA,NA,NA,NA,ORF2,2rd
1244,L1PA2,chrY,17060921,17066963,-,6042,1986,5825,NA,NA,NA,NA,NA,NA,NA,NA,ORF2,3rd


In [113]:
#data_ext5[-lines,] #好像应该删除重复的，而不应该把所有的都删掉；这里还没有做的很好；

In [114]:
data_ext6<-data_ext5[-lines,] #好像应该删除重复的，而不应该把所有的都删掉；这里还没有做的很好；

In [115]:
#应该删除的是，和3rd重复的行

In [116]:
dim(data_ext6)

[1] 12179    18

In [123]:
data_ext6[data_ext6$start=="17060921",]

,subfamily,chromsome,start,stop,strand,Length,orf.start,orf.end,%identity,alignment.length,mismatches,gap.opens,q.start,q.end,s.start,s. end,ORF,label
,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<chr>,<chr>
15113,L1PA2,chrY,17060921,17066963,-,6042,906,1922,97.64,1017,24,0,1,1017,1,1017,ORF1,null
1244,L1PA2,chrY,17060921,17066963,-,6042,1986,5825,NA,NA,NA,NA,NA,NA,NA,NA,ORF2,3rd


In [124]:
data_ext6[data_ext6$start=="77470642",]

,subfamily,chromsome,start,stop,strand,Length,orf.start,orf.end,%identity,alignment.length,mismatches,gap.opens,q.start,q.end,s.start,s. end,ORF,label
,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<chr>,<chr>
14262,L1PA5,chrX,77470642,77476783,+,6141,771,2033,89.577,1017,106,0,247,1263,1,1017,ORF1,null
18112,L1PA5,chrX,77470642,77476783,+,6141,2097,5317,NA,NA,NA,NA,NA,NA,NA,NA,ORF2,2rd


In [125]:
dim(data_ext6)[1]

[1] 12179

In [127]:
data_ext6[c(12178,12179),]

,subfamily,chromsome,start,stop,strand,Length,orf.start,orf.end,%identity,alignment.length,mismatches,gap.opens,q.start,q.end,s.start,s. end,ORF,label
,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<chr>,<chr>
1646,L1PA6,chr9,7277686,7283833,-,6147,2246,5931,NA,NA,NA,NA,NA,NA,NA,NA,ORF2,3rd
1647,L1PA6,chrX,55443238,55449415,+,6177,2108,4889,NA,NA,NA,NA,NA,NA,NA,NA,ORF2,3rd


In [128]:
lines2<-c()

In [129]:
for (m in 1:(dim(data_ext6)[1]-1)) {
if(sum(data_ext6[m,c(1:6,17)]==data_ext6[m+1,c(1:6,17)])==7){
    a=c(data_ext6[m,7],data_ext6[m,8],data_ext6[m+1,7],data_ext6[m+1,8])
    lines2<-append(lines2,m) #有一点小聪明
    lines2<-append(lines2,m+1)
    #data_ext6[i,]$label<-"1rd"
    #data_ext6[i+1,]$label<-"1rd"
    b=as.data.frame(c(data_ext6[m,1:6],min(a),max(a),rep(NA,8),data_ext6[m,17],"4rd"))
    colnames(b)<-colnames(data_ext6)
    data_ext6<-rbind(data_ext6,b)
    m = i+1
}
    }

In [130]:
#data_ext6[grepl("4rd", data_ext6$label),]

In [131]:
#tail(data_ext6,20)

In [132]:
data_ext6[data_ext6$start=="32228076",]

,subfamily,chromsome,start,stop,strand,Length,orf.start,orf.end,%identity,alignment.length,mismatches,gap.opens,q.start,q.end,s.start,s. end,ORF,label
,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<chr>,<chr>
1339,L1PA3,chr18,32228076,32233934,-,5858,3680,5641,NA,NA,NA,NA,NA,NA,NA,NA,ORF2,3rd
1341,L1PA3,chr18,32228076,32233934,-,5858,1974,5641,NA,NA,NA,NA,NA,NA,NA,NA,ORF2,3rd
1662,L1PA3,chr18,32228076,32233934,-,5858,1974,5641,NA,NA,NA,NA,NA,NA,NA,NA,ORF2,4rd


In [133]:
lines2

[1] 11949 11950 11972 11973 12041 12042 12080 12081

In [134]:
data_ext7<-data_ext6[-lines2,]

In [135]:
#data_ext7<-data_ext6[!grepl("3rd", data_ext6$label),] #这下就比较干净了；

In [136]:
#tail(data_ext7,20) #到4rd的时候，每一个4rd都不重复了

In [137]:
data_ext7[data_ext7$start=="32228076",]

,subfamily,chromsome,start,stop,strand,Length,orf.start,orf.end,%identity,alignment.length,mismatches,gap.opens,q.start,q.end,s.start,s. end,ORF,label
,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<chr>,<chr>
1662,L1PA3,chr18,32228076,32233934,-,5858,1974,5641,NA,NA,NA,NA,NA,NA,NA,NA,ORF2,4rd


In [212]:
#write.table(data_ext7,"data_ext7.txt",row.names=F,quote=F) #到这里，从序列整合的角度上来看，差不多了；#以上代码，不能重复运行，否则会爆出错误；

In [58]:
#table(data_ext7$label) #before


 2rd  3rd  4rd null 
2377  246    4 9693 

In [138]:
table(data_ext7$label)


 2rd  3rd  4rd null 
2474  270    4 9427 

In [214]:
#table(data_ext7$ORF) #before


 ORF1  ORF2 
 1147 11173 

In [140]:
table(data_ext7$ORF)


 ORF1  ORF2 
 1147 11028 

In [141]:
#先对data_ext7进行排序
data_ext8<-data_ext7[order(data_ext7$subfamily,data_ext7$chromsome,data_ext7$start,
                           data_ext7$stop,data_ext7$strand,data_ext7$Length,
                          data_ext7$ORF),]

In [142]:
write.table(data_ext8,"data_ext8_ordered.v2.txt",row.names=F,quote=F,sep="\t")

In [143]:
setwd("/home/xxzhang/workplace/software/ORFFinder/")
data_ext8<-read.table("data_ext8_ordered.v2.txt",sep="\t",header=T)

In [144]:
head(data_ext8,10)

,subfamily,chromsome,start,stop,strand,Length,orf.start,orf.end,X.identity,alignment.length,mismatches,gap.opens,q.start,q.end,s.start,s..end,ORF,label
,<chr>,<chr>,<int>,<int>,<chr>,<int>,<int>,<chr>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<chr>,<chr>
1,L1HS,chr1,103922066,103925398,+,3332,420,1334,96.831,915,29,0,1,915,103,1017,ORF1,null
2,L1HS,chr1,104770248,104776278,-,6030,907,1923,98.230,1017,18,0,1,1017,1,1017,ORF1,null
3,L1HS,chr1,104770248,104776278,-,6030,2281,5814,98.387,3534,57,0,1,3534,295,3828,ORF2,null
4,L1HS,chr1,104843834,104849864,-,6030,1986,5813,NA,NA,NA,NA,NA,NA,NA,NA,ORF2,2rd
5,L1HS,chr1,113341608,113345790,+,4182,1150,3962,NA,NA,NA,NA,NA,NA,NA,NA,ORF2,2rd
6,L1HS,chr1,113497221,113500000,+,2779,885,1901,97.837,1017,22,0,1,1017,1,1017,ORF1,null
7,L1HS,chr1,113497221,113500000,+,2779,1965,2777,98.893,813,9,0,1,813,1,813,ORF2,null
8,L1HS,chr1,113500001,113503257,+,3256,469,3012,98.153,2544,47,0,1,2544,1285,3828,ORF2,null
9,L1HS,chr1,116717925,116719731,+,1806,20,1588,98.853,1569,18,0,1,1569,2260,3828,ORF2,null


In [65]:
#这里就涉及到，我们怎样对这些L1s进行分类了。目前，可以将其按照有没有ORF，将其分成这四类
#only ORF1
#only ORF2
#both ORF1-ORF2
#not-enough-length

In [145]:
data_ext8[,"class"]<-"null"

In [146]:
head(data_ext8)

,subfamily,chromsome,start,stop,strand,Length,orf.start,orf.end,X.identity,alignment.length,mismatches,gap.opens,q.start,q.end,s.start,s..end,ORF,label,class
,<chr>,<chr>,<int>,<int>,<chr>,<int>,<int>,<chr>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<chr>,<chr>,<chr>
1,L1HS,chr1,103922066,103925398,+,3332,420,1334,96.831,915,29,0,1,915,103,1017,ORF1,null,null
2,L1HS,chr1,104770248,104776278,-,6030,907,1923,98.230,1017,18,0,1,1017,1,1017,ORF1,null,null
3,L1HS,chr1,104770248,104776278,-,6030,2281,5814,98.387,3534,57,0,1,3534,295,3828,ORF2,null,null
4,L1HS,chr1,104843834,104849864,-,6030,1986,5813,NA,NA,NA,NA,NA,NA,NA,NA,ORF2,2rd,null
5,L1HS,chr1,113341608,113345790,+,4182,1150,3962,NA,NA,NA,NA,NA,NA,NA,NA,ORF2,2rd,null
6,L1HS,chr1,113497221,113500000,+,2779,885,1901,97.837,1017,22,0,1,1017,1,1017,ORF1,null,null


In [28]:
dim(data_ext8) #before

[1] 12320    19

In [147]:
dim(data_ext8)

[1] 12175    19

In [69]:
# sum(data_ext8[10000,c(1:6)]==data_ext8[10000+1,c(1:6)])

In [70]:
# if(sum(c(NA,1,NA)==c(NA,1,NA))==6){
#     s=1
# } #找到原因了，一点点排查

In [71]:
#class(data_ext8)

In [72]:
#data_ext8[is.na(data_ext8$start),]
#data_ext8[is.na(data_ext8$alignment.length),]
#data_ext8[which(data_ext8$stop=="NA"),]
#data_ext8[which(data_ext8$stand=="NA"),]
#data_ext8[which(data_ext8$Length=="NA"),]
#data_ext8[which(data_ext8$orf.end=="NA"),]

In [73]:
#dim(data_ext8)

In [74]:
#1:dim(data_ext8)[1]

In [75]:
#data_ext8[1,17]

In [148]:
for (m in 1:(dim(data_ext8)[1]-1)) {
#print(m)
if(sum(data_ext8[m,c(1:6)]==data_ext8[m+1,c(1:6)])==6){
    #if(data_ext8[m,17]=="ORF1"& data_ext8[m+1,17]=="ORF2"){
        data_ext8[m,19]<-"ORF1-ORF2"
        data_ext8[m+1,19]<-"ORF1-ORF2" #注意区分
        #print(m)
        m = m+2 #即使程序运行完了加2，都无法改变1，2，3，4遍历的事实（想一下办法怎么改变这个局面）
        #print(m)
    #}
}  
}
    #print(m)

In [149]:
for (m in 1:(dim(data_ext8)[1]-1)) {
#print(m)
if(data_ext8[m,19]!="ORF1-ORF2"){
     if(data_ext8[m,17]=="ORF1"){
        data_ext8[m,19]<-"ORF1"
    }else{
        data_ext8[m,19]<-"ORF2"
}  
}
    }
#很好，一次解决不了的问题，那我们就分两次来解决;

In [150]:
head(data_ext8,16) #12连续出现两次，说明是紧接着的重复2次

,subfamily,chromsome,start,stop,strand,Length,orf.start,orf.end,X.identity,alignment.length,mismatches,gap.opens,q.start,q.end,s.start,s..end,ORF,label,class
,<chr>,<chr>,<int>,<int>,<chr>,<int>,<int>,<chr>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<chr>,<chr>,<chr>
1,L1HS,chr1,103922066,103925398,+,3332,420,1334,96.831,915,29,0,1,915,103,1017,ORF1,null,ORF1
2,L1HS,chr1,104770248,104776278,-,6030,907,1923,98.230,1017,18,0,1,1017,1,1017,ORF1,null,ORF1-ORF2
3,L1HS,chr1,104770248,104776278,-,6030,2281,5814,98.387,3534,57,0,1,3534,295,3828,ORF2,null,ORF1-ORF2
4,L1HS,chr1,104843834,104849864,-,6030,1986,5813,NA,NA,NA,NA,NA,NA,NA,NA,ORF2,2rd,ORF2
5,L1HS,chr1,113341608,113345790,+,4182,1150,3962,NA,NA,NA,NA,NA,NA,NA,NA,ORF2,2rd,ORF2
6,L1HS,chr1,113497221,113500000,+,2779,885,1901,97.837,1017,22,0,1,1017,1,1017,ORF1,null,ORF1-ORF2
7,L1HS,chr1,113497221,113500000,+,2779,1965,2777,98.893,813,9,0,1,813,1,813,ORF2,null,ORF1-ORF2
8,L1HS,chr1,113500001,113503257,+,3256,469,3012,98.153,2544,47,0,1,2544,1285,3828,ORF2,null,ORF2
9,L1HS,chr1,116717925,116719731,+,1806,20,1588,98.853,1569,18,0,1,1569,2260,3828,ORF2,null,ORF2


In [151]:
tail(data_ext8)

,subfamily,chromsome,start,stop,strand,Length,orf.start,orf.end,X.identity,alignment.length,mismatches,gap.opens,q.start,q.end,s.start,s..end,ORF,label,class
,<chr>,<chr>,<int>,<int>,<chr>,<int>,<int>,<chr>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<chr>,<chr>,<chr>
12170,L1PA8A,chrX,34343977,34346565,+,2588,662,1465,86.667,795,102,4,10,802,2095,2887,ORF2,null,ORF2
12171,L1PB1,chr14,51125216,51127997,-,2781,25,876,79.343,852,169,7,1,849,1285,2132,ORF2,null,ORF2
12172,L1PB1,chr2,151471244,151473198,-,1954,568,1953,93.290,1386,93,0,1,1386,1783,3168,ORF2,null,ORF2
12173,L1PB4,chr8,118531004,118536089,-,5085,2881,3702,94.526,822,43,2,1,822,773,1592,ORF2,null,ORF2
12174,L1PREC2,chr3,110671440,110676505,+,5065,2684,3754,91.511,1072,90,1,1,1071,427,1498,ORF2,null,ORF2
12175,L1PREC2,chrX,74758417,74760841,-,2424,248,1057,83.106,805,129,5,1,801,1837,2638,ORF2,null,null


In [42]:
#dim(data_ext8) #before

[1] 12320    19

In [152]:
dim(data_ext8)

[1] 12175    19

In [153]:
data_ext8[12175,19]<-"ORF2"

In [44]:
#暂时找不到原因，明天继续弄了；
#前面是ORF1-ORF2，那么重复的一定是ORF2，不会是ORF1

In [45]:
#table(data_ext8$class) #before


     ORF1 ORF1-ORF2      ORF2 
      132      2175     10013 

In [154]:
table(data_ext8$class) #这样就对了


     ORF1 ORF1-ORF2      ORF2 
      132      2030     10013 

In [84]:
12320-(132+1160+1160) #9868是ORF2的真实的片段的数目；

[1] 9868

In [155]:
2030+132+10013 #这样就对了，一点点小小的区别，就会导致结果的差异；

[1] 12175

In [158]:
data_ext8[data_ext8$class=="ORF1-ORF2",]

,subfamily,chromsome,start,stop,strand,Length,orf.start,orf.end,X.identity,alignment.length,mismatches,gap.opens,q.start,q.end,s.start,s..end,ORF,label,class
,<chr>,<chr>,<int>,<int>,<chr>,<int>,<int>,<chr>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<chr>,<chr>,<chr>
2,L1HS,chr1,104770248,104776278,-,6030,907,1923,98.230,1017,18,0,1,1017,1,1017,ORF1,null,ORF1-ORF2
3,L1HS,chr1,104770248,104776278,-,6030,2281,5814,98.387,3534,57,0,1,3534,295,3828,ORF2,null,ORF1-ORF2
6,L1HS,chr1,113497221,113500000,+,2779,885,1901,97.837,1017,22,0,1,1017,1,1017,ORF1,null,ORF1-ORF2
7,L1HS,chr1,113497221,113500000,+,2779,1965,2777,98.893,813,9,0,1,813,1,813,ORF2,null,ORF1-ORF2
10,L1HS,chr1,118852352,118858380,+,6028,905,1921,98.132,1017,19,0,1,1017,1,1017,ORF1,null,ORF1-ORF2
11,L1HS,chr1,118852352,118858380,+,6028,1985,5812,98.485,3828,58,0,1,3828,1,3828,ORF2,null,ORF1-ORF2
12,L1HS,chr1,121532231,121538261,-,6030,907,1923,98.033,1017,20,0,1,1017,1,1017,ORF1,null,ORF1-ORF2
13,L1HS,chr1,121532231,121538261,-,6030,1987,5811,NA,NA,NA,NA,NA,NA,NA,NA,ORF2,3rd,ORF1-ORF2
19,L1HS,chr1,174590324,174596379,-,6055,905,1921,98.132,1017,19,0,1,1017,1,1017,ORF1,null,ORF1-ORF2


In [312]:
#现在大致可以可以对每一个fragment的结构进行分类；
#在分类的基础之上，可以统计他们表达值的差异；
#以及每一个subfamily，各个类型所占的比例。

In [159]:
write.table(data_ext8,"data_ext8_ordered.final.txt",row.names=F,quote=F,sep="\t")

In [162]:
#按照家族，对数据进行计数
#data_ext8%>% count(subfamily,class,name = 'count') #好的，这个计数到了，我现在回顾一下原先的数据，看一下他们需要什么？

In [168]:
data_ext8$label<-paste(paste(paste(paste(data_ext8$chromsome,data_ext8$start,sep=":"),data_ext8$stop,sep="-"),data_ext8$strand,sep="("),"",sep=")")

In [172]:
data_ext8$label2<-paste(paste(paste(data_ext8$chromsome,data_ext8$start,sep=":"),data_ext8$stop,sep="-"),data_ext8$strand,sep=":")

In [174]:
head(data_ext8)

,subfamily,chromsome,start,stop,strand,Length,orf.start,orf.end,X.identity,alignment.length,mismatches,gap.opens,q.start,q.end,s.start,s..end,ORF,label,class,label2
,<chr>,<chr>,<int>,<int>,<chr>,<int>,<int>,<chr>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<chr>
1,L1HS,chr1,103922066,103925398,+,3332,420,1334,96.831,915,29,0,1,915,103,1017,ORF1,chr1:103922066-103925398(+),ORF1,chr1:103922066-103925398:+
2,L1HS,chr1,104770248,104776278,-,6030,907,1923,98.230,1017,18,0,1,1017,1,1017,ORF1,chr1:104770248-104776278(-),ORF1-ORF2,chr1:104770248-104776278:-
3,L1HS,chr1,104770248,104776278,-,6030,2281,5814,98.387,3534,57,0,1,3534,295,3828,ORF2,chr1:104770248-104776278(-),ORF1-ORF2,chr1:104770248-104776278:-
4,L1HS,chr1,104843834,104849864,-,6030,1986,5813,NA,NA,NA,NA,NA,NA,NA,NA,ORF2,chr1:104843834-104849864(-),ORF2,chr1:104843834-104849864:-
5,L1HS,chr1,113341608,113345790,+,4182,1150,3962,NA,NA,NA,NA,NA,NA,NA,NA,ORF2,chr1:113341608-113345790(+),ORF2,chr1:113341608-113345790:+
6,L1HS,chr1,113497221,113500000,+,2779,885,1901,97.837,1017,22,0,1,1017,1,1017,ORF1,chr1:113497221-113500000(+),ORF1-ORF2,chr1:113497221-113500000:+


In [175]:
write.table(data_ext8,"data_ext8_ordered.final.txt",row.names=F,quote=F,sep="\t")